In [ ]:
import csv
import random
import string
import time
import numpy as np

email_to_id = {}
paper_to_id = {}
HB = np.zeros((31, 28)) # (reviewers, papers) with entries in {-1, 0, 1}
MB = np.zeros((31, 28)) # (reviewers, papers) with entries in {-1, 0, 1}

reviewer_id = 0
with open('unanonymize/malicious_bidding.csv') as csvfile:
    csv_reader = csv.reader(csvfile)
    header = next(csv_reader)
    assert(header[11] == 'RecipientEmail' and header[17] == 'Q3_1' and header[44] == 'Q3_28')
    header = next(csv_reader)
    for paper_id, paper in enumerate(header[17:45]):
        paper_name = paper[paper.find(': -') + 3:].strip()
        paper_to_id[paper_name] = paper_id
    header = next(csv_reader)
    for row in csv_reader:
        email_to_id[row[11]] = reviewer_id
        for paper_id in range(28):
            if row[17 + paper_id] == 'Not willing to review':
                MB[reviewer_id, paper_id] = -1
            elif row[17 + paper_id] == 'Indifferent':
                MB[reviewer_id, paper_id] = 0
            elif row[17 + paper_id] == 'Eager to review':
                MB[reviewer_id, paper_id] = 1
        reviewer_id += 1

with open('unanonymize/honest_bidding.csv') as csvfile:
    csv_reader = csv.reader(csvfile)
    header = next(csv_reader)
    assert(header[11] == 'RecipientEmail' and header[17] == 'Q3_1' and header[44] == 'Q3_28')
    header = next(csv_reader)
    header = next(csv_reader)
    for row in csv_reader:
        if row[11] not in email_to_id:
            continue
        reviewer_id = email_to_id[row[11]]
        for paper_id in range(28):
            if row[17 + paper_id] == 'Not willing to review':
                HB[reviewer_id, paper_id] = -1
            elif row[17 + paper_id] == 'Indifferent':
                HB[reviewer_id, paper_id] = 0
            elif row[17 + paper_id] == 'Eager to review':
                HB[reviewer_id, paper_id] = 1


In [ ]:
paper_names = ["Making Machine Learning Fair",
"Interpreting AI Decision-Making",
"Transparent Machine-Learning Models",
"Voting for Participatory Budgeting",
"Voting with Delegation",
"Improved Learning from Rankings",
"Equilibrium Selection in Cooperative Games",
"Communication for Teamwork in Games",
"Multi-Agent Cooperative Board Games",
"An Overview of Zero-Sum Games",
"Solution Concepts in Many-Player Games",
"Stackelberg Security Games for Coastal Defense",
"A No-Regret Algorithm for Efficient Equilibrium Computation",
"Consistency of Bayesian Inference",
"Fitting Graphical Models",
"A Novel Gaussian Process Approximation",
"A*+DFS: A Hybrid Search Algorithm",
"A* Search Under Uncertainty",
"Search-Based Planning of Robot Trajectories",
"Online Convex Optimization with Regularization",
"Private Stochastic Convex Optimization",
"A Hybrid Method for Non-Convex Optimization",
"Memory and Computation-Efficient Kernel SVMs",
"Forecasting Stock Prices with Deep Learning",
"A Deep Learning Approach for Anomaly Detection",
"New Algorithms for 3D Computer Vision",
"Towards More Accurate NLP Models",
"Optimal Error Bounds in Statistical Learning Theory"]

reviewer_to_sas = {} # dict of review -> list of subject areas
reviewer_to_group = {} # dict of reviewer -> group id
reviewer_to_sa = {} # dict of review -> subject area of authored paper
sa_to_paper_ids = {} # dict of subject area -> list of papers in the subject area
paper_to_sa = {} # dict of paper -> subject area
group_to_targets = {} # dict of group -> list of target papers

SA = np.zeros(HB.shape) # (reviewers, papers) with entries in {0, 1}
group_map = {} # dict of group id -> list of reviewer indices in that group
author_map = {} # dict of reviewer -> index of paper authored
target_map = {} # dict of reviewer -> index of target paper (for reviewers with group size 1)
author_map_group = {} # dict of reviewer -> indices of target papers (including other team members' authored papers)

with open('analysis/new_groups.csv') as csvfile:
    r = csv.reader(csvfile)
    header = next(r)
    assert(all([x == y for x, y in 
        zip(header, ['name', 'sas', 'authored_sa', 'authored_id', 'target_sa', 'target_id', 'group'])
        ]))
    for row in r:
        if row[0] in email_to_id:
            reviewer_id = email_to_id[row[0]]
        else:
            reviewer_id = -1
        
        sas = [int(x) for x in row[1].strip().split(' ')]
        if reviewer_id >= 0:
            reviewer_to_sas[reviewer_id] = sas
        
        sa_id = int(row[2])
        paper_id = paper_to_id[paper_names[int(row[3])]]
        if reviewer_id >= 0:
            reviewer_to_sa[reviewer_id] = sa_id
            author_map[reviewer_id] = paper_id          
        paper_to_sa[paper_id] = sa_id
        if sa_id in sa_to_paper_ids:
            sa_to_paper_ids[sa_id].append(paper_id)
        else:
            sa_to_paper_ids[sa_id] = [paper_id]
        
        if reviewer_id >= 0:
            group = int(row[6])
            if group in group_map:
                group_map[group].append(reviewer_id)
            else:
                group_map[group] = [reviewer_id]
            reviewer_to_group[reviewer_id] = group
            if len(row[5]) > 0:
                target_id = paper_to_id[paper_names[int(row[5])]]
                target_map[reviewer_id] = target_id
        
        group = int(row[6])
        if group in group_to_targets:
            group_to_targets[group].append(paper_id)
        else:
            group_to_targets[group] = [paper_id]

for reviewer_id in range(len(reviewer_to_sas)):
    for sa_id in reviewer_to_sas[reviewer_id]:
        for paper_id in sa_to_paper_ids[sa_id]:
            SA[reviewer_id, paper_id] = 1

for reviewer_id in range(len(reviewer_to_group)):
    author_map_group[reviewer_id] = group_to_targets[reviewer_to_group[reviewer_id]]

In [ ]:
import pickle

with open('analysis/Biddings.npy', 'wb') as f:
    np.save(f, HB)
    np.save(f, MB)
    np.save(f, SA)

with open('analysis/maps.pkl', 'wb') as f:
    pickle.dump(author_map, f)
    pickle.dump(group_map, f)
    pickle.dump(target_map, f)
    pickle.dump(author_map_group, f)


In [ ]:
with open('analysis/Result.npy', 'rb') as f:
    success_by_reviewer = np.load(f)
    rank_by_reviewer_simple = np.load(f)
    rank_by_reviewer_cluster = np.load(f)
    rank_by_reviewer_row_rank = np.load(f)
    diff_by_reviewer_hamming = np.load(f)
    diff_by_reviewer_L1 = np.load(f)

email_list = list(email_to_id.keys())
reviewer_id_list = list(email_to_id.values())
     
with open('analysis/Result.csv', 'w', newline='') as csvfile:
    column_names = ['Email', 'Group_id', 'Collaborators', 'Success_rate', 'Rank_simple', \
                   'Rank_cluster', 'Rank_low_rank', 'Diff_hamming', 'Diff_L1']
    csv_writer = csv.DictWriter(csvfile, fieldnames=column_names)
    csv_writer.writeheader()
    for reviewer_idx in range(success_by_reviewer.shape[0]):
        row = {}
        row['Email'] = email_list[reviewer_id_list.index(reviewer_idx)]
        row['Group_id'] = reviewer_to_group[reviewer_idx]
        row['Collaborators'] = group_map[row['Group_id']]
        row['Success_rate'] = success_by_reviewer[reviewer_idx]
        row['Rank_simple'] = rank_by_reviewer_simple[reviewer_idx]
        row['Rank_cluster'] = rank_by_reviewer_cluster[reviewer_idx]
        row['Rank_low_rank'] = rank_by_reviewer_row_rank[reviewer_idx]
        row['Diff_hamming'] = diff_by_reviewer_hamming[reviewer_idx]
        row['Diff_L1'] = diff_by_reviewer_L1[reviewer_idx]
        csv_writer.writerow(row)

In [ ]:
def count(li_):
    li = list(li_)
    ctr = [0, 0, 0, 0, 0, 0, 0]
    for x in li:
        if 0 <= x < 2:
            ctr[0] += 1
        if 2 <= x < 5:
            ctr[1] += 1
        if 5 <= x < 9:
            ctr[2] += 1
        if 9 <= x < 13:
            ctr[3] += 1
        if 13 <= x < 15:
            ctr[4] += 1
        if 15 <= x < 19:
            ctr[5] += 1
        if 19 <= x < 25:
            ctr[6] += 1
    return ctr

print(count(reviewer_to_sa.values()))
print(count(paper_to_sa.values()))
print(group_to_targets)